# Default building control setting up an empty action interface

When you want to run a simulation with all the default building control (specified in the building file). You can directly use the **EnergyPlus simulation engine**. For example, placing us in the workspace of the container would be to run the following:

```bash
$ energyplus -w sinergym/data/weather/USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw sinergym/data/buildings/5ZoneAutoDXVAV.epJSON
```

However, doing so without our framework has some **disadvantages**. You will have EnergyPlus default output and will not have some **added outputs** such as our wrapper logger that monitors all interactions with the environment. The buildings have a default *Site:Location* and *SizingPeriod:DesignDay*, which Sinergym changes automatically depending on the specified weather, so you would have to **adjust it** before using the simulator manually. Finally, the *RunPeriod* of the buildings are set to one episode for DRL, which means that as the buildings stand you can only simulate one year. So, you would also have to modify the *RunPeriod* **manually** in the building file before starting the simulation.

Therefore, our recommended proposal is setting up an **empty action interface** in an environment with Sinergym. For example:

In [2]:
import gymnasium as gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import LoggerWrapper

env = gym.make(
    'Eplus-office-hot-continuous-v1',
    actuators={},
    action_space=gym.spaces.Box(
        low=0,
        high=0,
        shape=(0,)))
env = LoggerWrapper(env)

for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [office-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 6, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment office-hot-continuous-v1 created successfully.
[WRAPPER LoggerWrapper] (INFO) : Wrapper initialized.
#---------------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1/Eplus-env-sub_run1/output', '/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1/Eplus-env-sub_run1/ASHRAE901_OfficeMedium_STD2019_Denver.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 1) if logger is active
Reward:  -1.040098962863255 {'time_elapsed(hours)': 0.75, 'year': 2023, 'month': 7, 'day': 21, 'hour': 0, 'is_raining': False, 'action': [], 'timestep': 2, 'reward': -1.040098962863255, 'reward_energy': -2.037786761565728, 'reward_comfort': -0.04241116416078228, 'total_energy': 20377.86761565728, '

In this example, a default environment is loaded, but the space and definition of the default action **is replaced with an empty one**, Sinergym takes care of making the necessary changes in the background. Then, the random agent implemented send **empty actions ([])**.

The advantages are that you can **combine the weathers with the buildings** as you want, Sinergym will take care of adapting everything automatically (you don't have the disadvantages of before), you can run in a single experiment as many years as you want, with our loggers. When you set an empty action interface, Sinergym preserves the default actuators that the building definition comes with (these can be more or less sophisticated depending on the definition of the building in the *epJSON* file).